# Machine Learning Project

## Project description

Project description
Mobile carrier Megaline has found out that many of their subscribers use legacy plans. They want to develop a model that would analyze subscribers' behavior and recommend one of Megaline's newer plans: *Smart* or *Ultra*.
We have access to behavior data about subscribers who have already switched to the new plans (from the project for the Statistical Data Analysis course). For this classification task, you need to develop a model that will pick the right plan with the highest possible accuracy.

## Opening and investigating the data

In [2]:
#making sure I and the reviewer have the same versions
#!pip install -U matplotlib
#!pip install -U numpy
#!pip install -U pandas
#!pip install -U scipy

In [3]:
import warnings
#warnings.filterwarnings('ignore')

In [76]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import numpy as np
#import math
#from scipy import stats as st

In [5]:
try:
    # If we are online on the Jupyter Hub, we open the online database
    behavior_data=pd.read_csv("/datasets/users_behavior.csv")
except:
    # If we are locally we open the local database
    behavior_data=pd.read_csv("dataset/users_behavior.csv")

### Checking data

In [6]:
print(behavior_data.shape)
behavior_data.info()
print(behavior_data.describe())
behavior_data

(3214, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%      40.000000   274.575000     9.000000  12491.902500     0.000000
50%      62.000000   430.600000    30.000000  16943.235000     0.000000
75%      82.000000   571.927500    57.000000  21424.700

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


### Conclusion

We see no obvious flaws with the database. We thank the Megaline company for providing us quality data :)

## Checking our features
It is clear that our target for classification is the `is_ultra` column.
Before splitting the data we want to see how much are the rest of the columns influencing our target. This can help us decide which features we need to take into account. For this we use a correlation table.

In [97]:
behavior_data.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


We see that each feature, calls, minutes, messages and mb used have an approximately equal (and pretty small) part in influencing our result, about 0.2.

## Splitting the data

The source data has to be split into three parts: training, validation, and test. A common ratio for such a split is  6:2:2 ratio. Another possible ration is 8:1:1. In order to see which ratio give the best results, we will try both ratios. Using `train_test_split` we first split into 6:4 and 8:2 train and temp sets and then the temp set we further split it into equal validation and test sets.

In [11]:
X = behavior_data.drop(columns = ['is_ultra']).copy()
y = behavior_data['is_ultra']

# In the first step we will split the data in training and remaining dataset
X_train6, X_temp, y_train6, y_temp = train_test_split(X,y, train_size=0.6)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid2, X_test2, y_valid2, y_test2 = train_test_split(X_temp,y_temp, test_size=0.5)

# We do the same thing for the 8:1:1 split
X_train8, X_temp, y_train8, y_temp = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid1, X_test1, y_valid1, y_test1 = train_test_split(X_temp,y_temp, test_size=0.5)

print('The sets for the 6:2:2 split have shapes:',X_train6.shape, y_train6.shape,
          X_valid2.shape, y_valid2.shape,
          X_test2.shape, y_test2.shape)
print('The sets for the 8:1:1 split have shapes:',X_train8.shape, y_train8.shape,
          X_valid1.shape, y_valid1.shape,
          X_test1.shape, y_test1.shape)

The sets for the 6:2:2 split have shapes: (1928, 4) (1928,) (643, 4) (643,) (643, 4) (643,)
The sets for the 8:1:1 split have shapes: (2571, 4) (2571,) (321, 4) (321,) (322, 4) (322,)


## Checking the Classification Models

We create a DataFrame which will hold the accuracy of the varios models and splits we will try.

In [121]:
result_models=pd.DataFrame(index=['DTC','RFC','LR'],columns=['622','811'])

### Decision Tree Classifier

In [127]:
def dtc(X_train,y_train,X_valid,y_valid,X_test,y_test):
    best_depth=0;
    best_acc=0;

    for depth in range(1,6):
        model =  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=depth,
                max_features=None, max_leaf_nodes=None,
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, random_state=12345,
                splitter='best')
        model.fit(X_train, y_train)

        predictions_valid = model.predict(X_valid)# < find the predictions using validation set >

        print('max_depth =',depth,': ',end='')
        acc=accuracy_score(y_valid, predictions_valid)
        print(acc)
        if acc>best_acc :
            best_acc=acc
            best_depth=depth
            best_model=model
    print('Best depth in training test =',best_depth,'. ')
    predictions=best_model.predict(X_test)
    acc=accuracy_score(y_test, predictions)
    print('Accuracy at this depth in the validation test:',acc)
    return acc

In [128]:
print('For the 6:2:2 split we get the folowing accuracy: ',end='')
result_models.loc['DTC','622']=dtc(X_train6,y_train6,X_valid2,y_valid2,X_test2,y_test2)
print('For the 8:1:1 split we get the folowing accuracy: ',end='')
result_models.loc['DTC','811']=dtc(X_train8,y_train8,X_valid1,y_valid1,X_test1,y_test1)

For the 6:2:2 split we get the folowing accuracy: max_depth = 1 : 0.7527216174183515
max_depth = 2 : 0.776049766718507
max_depth = 3 : 0.7869362363919129
max_depth = 4 : 0.776049766718507
max_depth = 5 : 0.7822706065318819
Best depth in training test = 3 . 
Accuracy at this depth in the validation test: 0.7853810264385692
For the 8:1:1 split we get the folowing accuracy: max_depth = 1 : 0.7445482866043613
max_depth = 2 : 0.7601246105919003
max_depth = 3 : 0.7819314641744548
max_depth = 4 : 0.7850467289719626
max_depth = 5 : 0.7975077881619937
Best depth in training test = 5 . 
Accuracy at this depth in the validation test: 0.8012422360248447


We see that the 8:1:1 split give better results with the Decision Tree Classifier.

### Random Forest Classifier

In [142]:
def rfc(X_train,y_train,X_valid,y_valid,X_test,y_test):
    # < create a loop for max_depth from 1 to 10 >
    best_est=0;
    best_acc=0;
    for n_est in range(1,11):

        model = RandomForestClassifier( n_estimators=n_est)
        model.fit(X_train, y_train)

        print('n_estimators =',n_est,': ',end='')
        acc=model.score(X_valid, y_valid)
        print(acc) 
        if acc>best_acc :
            best_acc=acc
            best_est=n_est
            best_model=model
    print('Best number of estimators applied on validation set =',best_est,': ',end='')
    acc=best_model.score(X_test, y_test)
    print(acc)
    return(acc)

In [143]:
print('For the 6:2:2 split we get the folowing reuslts:')
result_models.loc['RFC','622']=rfc(X_train6,y_train6,X_valid2,y_valid2,X_test2,y_test2)
print('For the 8:1:1 split we get the folowing reuslts:')
result_models.loc['RFC','811']=rfc(X_train8,y_train8,X_valid1,y_valid1,X_test1,y_test1)

For the 6:2:2 split we get the folowing reuslts:
n_estimators = 1 : 0.7278382581648523
n_estimators = 2 : 0.7465007776049767
n_estimators = 3 : 0.76049766718507
n_estimators = 4 : 0.7573872472783826
n_estimators = 5 : 0.7636080870917574
n_estimators = 6 : 0.7900466562986003
n_estimators = 7 : 0.7853810264385692
n_estimators = 8 : 0.7822706065318819
n_estimators = 9 : 0.7869362363919129
n_estimators = 10 : 0.7978227060653188
Best number of estimators applied on validation set = 10 : 0.8040435458786936
For the 8:1:1 split we get the folowing reuslts:
n_estimators = 1 : 0.67601246105919
n_estimators = 2 : 0.735202492211838
n_estimators = 3 : 0.7383177570093458
n_estimators = 4 : 0.7663551401869159
n_estimators = 5 : 0.7258566978193146
n_estimators = 6 : 0.8006230529595015
n_estimators = 7 : 0.7757009345794392
n_estimators = 8 : 0.7881619937694704
n_estimators = 9 : 0.7663551401869159
n_estimators = 10 : 0.7757009345794392
Best number of estimators applied on validation set = 6 : 0.8260869

We see that the 8:1:1 split give better results with the Random Forest Classifier.

### Logistics Regression

Since in Logistics Regression we do not make use of both a validation and test data, we will add the unused data back to the training set

In [148]:
def lr(X_train,y_train,X_valid,y_valid,X_test,y_test):
    model = LogisticRegression(random_state=12345, solver='lbfgs') 
    model.fit(X_train.append(X_test), y_train.append(y_test)) 
    acc=model.score(X_valid, y_valid)
    print( acc)
    return acc

In [149]:
print('For the 6:2:2 split we get the folowing accuracy: ',end='')
result_models.loc['LR','622']=lr(X_train6,y_train6,X_valid2,y_valid2,X_test2,y_test2)
print('For the 8:1:1 split we get the folowing accuracy: ',end='')
result_models.loc['LR','811']=lr(X_train8,y_train8,X_valid1,y_valid1,X_test1,y_test1)

For the 6:2:2 split we get the folowing accuracy: 0.744945567651633
For the 8:1:1 split we get the folowing accuracy: 0.7414330218068536


We see that neither split give a satisfying result (an accuracy better than the target of 75%).

### Sanity Check

In [99]:
behavior_data[behavior_data.is_ultra==1]

,calls,minutes,messages,mb_used,is_ultra
3,106.0,745.53,81.0,8437.39,1
6,57.0,431.64,20.0,3738.90,1
8,7.0,43.39,3.0,2538.67,1
10,82.0,560.51,20.0,9619.53,1
14,108.0,587.90,0.0,14406.50,1
...,...,...,...,...,...
3201,56.0,419.42,59.0,5177.62,1
3203,53.0,390.39,85.0,30550.30,1
3208,164.0,1016.98,71.0,17787.52,1
3209,122.0,910.98,20.0,35124.90,1


In [100]:
len(behavior_data[behavior_data.is_ultra==1])/ len(behavior_data)

0.30647168637212197

About 30% of the subscribers are in the Ultra program.
Common sense would say that the top 30% of users are in the Ultra program. Let's see how this fare against the Machine Learning algorythms

In [101]:
behavior_data.describe(percentiles=[.3, .7])

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
30%,45.000000,309.982000,14.000000,13637.495000,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
70%,78.000000,537.534000,50.000000,20397.059000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


We see that every user that talked more than 537.53 minutes is in the top 30%

In [102]:
behavior_data['sanity_check']=(behavior_data['minutes']>537.53)*1

In [103]:
behavior_data

,calls,minutes,messages,mb_used,is_ultra,sanity_check
0,40.0,311.90,83.0,19915.42,0,0
1,85.0,516.75,56.0,22696.96,0,0
2,77.0,467.66,86.0,21060.45,0,0
3,106.0,745.53,81.0,8437.39,1,1
4,66.0,418.74,1.0,14502.75,0,0
...,...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1,1
3210,25.0,190.36,0.0,3275.61,0,0
3211,97.0,634.44,70.0,13974.06,0,1
3212,64.0,462.32,90.0,31239.78,0,0


We need to calculate the accuracy of the simple model we made. We use the following formula:
$$accuracy=\frac{total\,number\,of\, events - mistaken\, guesses}{total\, number\, of\, events}$$

To get the number of incorrect guesses all we have to do is apply XOR between the `sanity_check` column and the `is_ultra` column and count the number of not null results.

In [104]:
behavior_data['mistaken']=np.bitwise_xor(behavior_data['is_ultra'],behavior_data['sanity_check'])
mistaken_guesses=behavior_data['mistaken'].sum()
mistaken_guesses

1060

In [105]:
sanity_check_accuracy=(len(behavior_data)-mistaken_guesses)/len(behavior_data)
sanity_check_accuracy

0.670192906036092

We get an accuracy of just $67\%$ which is lower than all the other models examined.

## Overall Conclusion
We have opened and explored the table. 
The data didn't require processing.
We checked the correlation table to select our features that will be used in the learning models. We saw that each feature contributed relatively equally to the target.
We split the data both in the 6:2:2 ratio and the 8:1:1 ratio, to understand which ratio would give better results.

We checked three models: **Decision Tree Classifier**, **Random Forest Classifier** and **Logistic Regression**. This is the tabel with the results we received.

In [150]:
result_models

,622,811
DTC,0.785381,0.801242
RFC,0.804044,0.826087
LR,0.744946,0.741433


We see that all models got a better result than the sanity check model. Logistics Regression didn't succeed in giving a better accuracy than the minimal target which was 75%. (It is worth noting that the Logistics Regression split was actually 8:2 and 9:1. Even with extra training data this didn't improve enough the result)
We see in most cases the 8:1:1 ratio gave better results. More training data gives better Learning Models.

The overal winner is: **Random Forest Classifier** with a 8:1:1 split. This gives an **0.826 accuracy**!